In [1]:
# !unzip robohearts.zip
#%cd robohearts/

In [1]:
import gym
import multiprocessing
from gymhearts.Hearts import *
from gymhearts.Agent.human import Human
from gymhearts.Agent.random_agent import RandomAgent
from gymhearts.Agent.monte_carlo_nn import MonteCarloNN
from gymhearts.Agent.agent_utils import save_model
from tqdm import tqdm_notebook

In [4]:
NUM_TESTS = 25
NUM_EPISODES = 1000
TRAINING_ITERS = 10000
MAX_SCORE = 100

run_train = True

# in_hand, in_play, played_cards, won_cards, scores
feature_list = ['in_hand', 'in_play']

playersNameList = ['MonteCarlo', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

# Human vs Random
mc_config = {
    'print_info' : False,
    'epsilon' : .05,
    'gamma' : 1.0,
    'alpha': 5e-4,
    'feature_list' : feature_list
}
agent_list[0] = MonteCarloNN(playersNameList[0], mc_config)
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

In [3]:
# TRAIN THE MONTE CARLO AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
if run_train:
    for trn_episode in tqdm_notebook(range(TRAINING_ITERS)):
        observation = env.reset()
        history = []
        while True:
            #env.render()

            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            # update my agent before clearing state
            if now_event == 'RoundEnd':
                agent_list[0].update_weights(history, -reward['MonteCarlo'])
                history = []
            if now_event == 'GameOver':
                  break
            if observation['event_name']=='PlayTrick' and observation['data']['playerName'] == 'MonteCarlo':
                # don't add score, they don't change till at end of round
                history.append((observation, agent_list[0].played_cards, agent_list[0].won_cards))

            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)

            observation, reward, done, info = env.step(action)
    save_model(agent_list[0].nn)

In [5]:
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'MonteCarlo')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [6]:
# EVALUATE THE MONTE CARLO AGENT
    
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
mc_config = {
    'print_info' : False,
    'nn_path' : 'mlp',
    'feature_list' : feature_list
}
agent_list[0] = MonteCarloNN(playersNameList[0], mc_config)
mc_wins = [0] * NUM_TESTS        
pool = multiprocessing.Pool(processes=NUM_TESTS)
mc_wins = pool.map(run_test, mc_wins)
print(mc_wins)
pool.close()
pool.join()

loaded from /home/akashkw/repos/robohearts/gymhearts/Agent/mlp.th
                         


























[32, 30, 35, 26, 22, 27, 29, 24, 33, 31, 36, 23, 19, 28, 26, 27, 31, 29, 30, 34, 29, 34, 27, 34, 29]


In [ ]:
def run_test_2(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'Randman')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [ ]:
# EVALUATE THE RANDOM AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
playersNameList[0] = 'Randman'
agent_list[0] = RandomAgent(playersNameList[0])
rand_wins = [0] * NUM_TESTS
        
pool = multiprocessing.Pool(processes=NUM_TESTS)
rand_wins = pool.map(run_test_2, rand_wins)
print(rand_wins)
pool.close()
pool.join()

In [ ]:
print(f"Monte Carlo won {sum(mc_wins)/len(mc_wins)} times on average :: {str(mc_wins)}")
print(f"Random won {sum(rand_wins)/len(rand_wins)} times on average :: {str(rand_wins)}")
# print(f"\n\nThe Monte Carlo weights are: {str(weights)}")